### Churn @ Robinhood
#### Load Libraries and Data

In [1]:
from numba import cuda

try:
    print("Available GPUs:", cuda.gpus)
except cuda.CudaSupportError as e:
    print("CUDA Error:", e)

Available GPUs: CUDA Error: Error at driver init: Call to cuInit results in CUDA_ERROR_UNKNOWN (999)


In [2]:
import cudf as cf
equity_df_raw = cf.read_csv('./data/equity_value_data.csv')
features_df_raw = cf.read_csv('./data/features_data.csv')

equity_df = equity_df_raw.copy()
features_df = features_df_raw.copy()

/home/oem/Documents/github/magnimind_projects/magpenv/lib/python3.12/site-packages/cudf/utils/_ptxcompiler.py:64: UserWarning: Error getting driver and runtime versions:

stdout:



stderr:

Traceback (most recent call last):
  File "/home/oem/Documents/github/magnimind_projects/magpenv/lib/python3.12/site-packages/numba_cuda/numba/cuda/cudadrv/driver.py", line 254, in ensure_initialized
    self.cuInit(0)
  File "/home/oem/Documents/github/magnimind_projects/magpenv/lib/python3.12/site-packages/numba_cuda/numba/cuda/cudadrv/driver.py", line 304, in safe_cuda_api_call
    self._check_ctypes_error(fname, retcode)
  File "/home/oem/Documents/github/magnimind_projects/magpenv/lib/python3.12/site-packages/numba_cuda/numba/cuda/cudadrv/driver.py", line 372, in _check_ctypes_error
    raise CudaAPIError(retcode, msg)
numba.cuda.cudadrv.driver.CudaAPIError: [999] Call to cuInit results in CUDA_ERROR_UNKNOWN

During handling of the above exception, another exception occurred:

Traceback (most 

CUDARuntimeError: cudaErrorUnknown: unknown error

In [ ]:
equity_df.info()

In [ ]:
features_df.info()

#### a). What percentage of users have churned in the data?
A user is *churned* when their equity falls below 10 usd for 28 consecutive calendar days or longer having perviously been at least 10 usd

**NOTE** Since no equities falls under 10 usd, threshold is set to a variable usd instead

In [ ]:
# Set threshold
thresh = 12

# Step 1: Ensure the timestamp column is datetime and sort the data
equity_df['timestamp'] = cf.to_datetime(equity_df['timestamp'])
equity_df = equity_df.sort_values(['user_id', 'timestamp'])

# Step 2: Flag close_equity below threshold
equity_df[f'below_{thresh}'] = (equity_df['close_equity'] < thresh).astype(int)

# Step 3: Compute rolling 28-day windows for each user
equity_df[f'below_{thresh}_28d'] = equity_df.groupby('user_id')[f'below_{thresh}'].rolling(window=28, min_periods=28).sum().reset_index(0, drop=True)

# Step 4: Identify churn (continuous 28 days below $11)
equity_df['churn'] = (equity_df[f'below_{thresh}_28d'] == 28).astype(int).copy()

# Step 5: Check if the user ever had close_equity >= 11
# Group by 'user_id' to find the max close_equity
user_max_equity = equity_df.groupby('user_id')['close_equity'].max().reset_index()
user_max_equity.rename(columns={"close_equity": "max_equity"}, inplace=True)

# Merge back to associate max_equity with each user_id in the main DataFrame
equity_df = equity_df.merge(user_max_equity, on="user_id", how="left")

# Add a flag for users who had close_equity >= 11 at some point
equity_df[f'above_{thresh}_before'] = (equity_df['max_equity'] >= thresh).astype(int)

# Step 6: Filter churned users
churned_users = equity_df.loc[
    (equity_df['churn'] == 1) & (equity_df[f'above_{thresh}_before'] == 1),
    'user_id'
].unique()

# Step 7: Calculate the churn percentage
total_users = equity_df['user_id'].nunique()
churn_percentage = (len(churned_users) / total_users) * 100

print(f"Churned Percentage: {churn_percentage:.2f}%\n")
print(f"Churned Users: {churned_users}\n")
print(f"Total Users: {total_users}\n")

b). Build a classifier given a user with their features assigns a churn probability for every user and predicts which users will churn

In [ ]:
# Create the 'churned' column
features_df['churned'] = features_df['user_id'].isin(churned_users).astype('int32')
features_df['churned'].value_counts()

In [ ]:
features_df.info()

In [ ]:
features_df.describe().T

In [ ]:
features_df['risk_tolerance'].value_counts()

In [ ]:
features_df['investment_experience'].value_counts()

In [ ]:
# change features of investment experience to 3
features_df['investment_experience'] = features_df['investment_experience'].replace({
    'extensive_investment_exp': 'good_investment_exp'
})
features_df['investment_experience'].value_counts()

In [ ]:
features_df['liquidity_needs'].value_counts()

In [ ]:
features_df['platform'].value_counts()

In [ ]:
features_df['instrument_type_first_traded'].value_counts()

In [ ]:
# Use vectorized operations to assign 'non_stock' to all values not equal to 'stock'
features_df['instrument_type_first_traded'] = (
    features_df['instrument_type_first_traded']
    .where(features_df['instrument_type_first_traded'] == 'stock', 'non_stock')
)

# Check the updated value counts
print(features_df['instrument_type_first_traded'].value_counts())

In [ ]:
features_df['time_horizon'].value_counts()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

features_df = features_df.to_pandas()

# Drop `user_id` and `churned` columns for training
features_df = features_df.drop(columns=['user_id'])

# Identify categorical and numerical features
categorical_features = ['risk_tolerance', 'investment_experience', 'liquidity_needs', 'platform', 
                        'instrument_type_first_traded', 'time_horizon']
numerical_features = ['time_spent', 'first_deposit_amount']

# Define preprocessing for numerical and categorical features
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(sparse_output=False, handle_unknown='ignore', min_frequency=0.01)

# Combine preprocessors in a column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Define anomaly detection models and parameter grids
param_grids = {
    'IsolationForest': {
        'model__contamination': [0.005, 0.01, 0.02, 0.05],
        'model__max_samples': [128, 256, 'auto'],
        'model__max_features': [0.25, 0.5, 0.75, 1.0]
    },
    'LocalOutlierFactor': {
        'model__n_neighbors': [10, 20, 35, 50],
        'model__leaf_size': [10, 25, 30, 50],
        'model__contamination': [0.005, 0.01, 0.02, 0.05]
    },
    'OneClassSVM': {
        'model__nu': [0.005, 0.01, 0.05, 0.1],
        'model__gamma': ['scale', 'auto'],
        'model__kernel': ['rbf', 'poly', 'linear']
    }
}

true_labels = features_df['churned']  # Ground truth labels for evaluation

for name, param_grid in param_grids.items():
    if name == 'IsolationForest':
        model = IsolationForest(random_state=42)
    elif name == 'LocalOutlierFactor':
        model = LocalOutlierFactor(novelty=True)
    elif name == 'OneClassSVM':
        model = OneClassSVM()

    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    grid_search = GridSearchCV(
        pipeline,
        param_grid,
        scoring='f1_macro',  # Adjust scoring based on your metric of interest
        cv=3
    )

    grid_search.fit(features_df.drop(columns=['churned']), true_labels)
    best_pipeline = grid_search.best_estimator_

    # Predict using the best pipeline
    predictions = best_pipeline.predict(features_df.drop(columns=['churned']))
    binary_predictions = (predictions == 1).astype(int)

    # Print classification report
    print(f"\n{name} Best Parameters: {grid_search.best_params_}\n")
    print(f"{name} Classification Report:\n")
    print(classification_report(true_labels, binary_predictions))

    # Print feature importances if available
    if hasattr(best_pipeline.named_steps['model'], 'feature_importances_'):
        feature_importances = best_pipeline.named_steps['model'].feature_importances_
        feature_names = numerical_features + list(best_pipeline.named_steps['preprocessor']
                                                  .transformers_[1][1]
                                                  .get_feature_names_out(categorical_features))
        importance_df = pd.DataFrame({
            'Feature': feature_names,
            'Importance': feature_importances
        }).sort_values(by='Importance', ascending=False)
        print(f"\n{name} Feature Importances:\n")
        print(importance_df)
